In [ ]:
import pandas as pd
import bibtexparser # Used to read .bib files
import re
pd.set_option("display.max_columns", None)

In [ ]:

ieee_df = pd.read_csv("dataset/IEEE.csv")
ieee_df



In [ ]:
with open("dataset/acm.bib", encoding="utf-8") as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

# Convert entries to a pandas DataFrame
acm_df = pd.DataFrame(bib_database.entries)
acm_df



In [ ]:
springer_df = pd.read_parquet("dataset/springer.parquet")
springer_df

In [ ]:
# read pubmed parquet
pubmed_df = pd.read_parquet("dataset/pubmed.parquet")

pubmed_df

In [ ]:
sciencedirect_df = pd.read_parquet("dataset/sciencedirect.parquet")
elsevier_df = sciencedirect_df
elsevier_df

In [ ]:
# 1. Create a list of tuples: (dataframe_name, title_series)
df_sources = [
    ("ACM", acm_df["title"]),
    ("Elsevier", elsevier_df["title"]),
    ("IEEE", ieee_df["Document Title"]),
    ("PubMed", pubmed_df["Title"]),
    ("Springer", springer_df["Item Title"])
]

# 2. Create a combined DataFrame with source information
combined_data = []
for source_name, title_series in df_sources:
    # Create a temporary DataFrame with title and source
    temp_df = pd.DataFrame({
        'title': title_series.str.lower(),  # Normalize to lowercase
        'source': source_name
    })
    combined_data.append(temp_df)

# Combine all into one DataFrame
all_titles_df = pd.concat(combined_data, ignore_index=True)
print(f"Amount of papers direct (from all libraries) directly after downloading / boolean queries:: {len(all_titles_df)}")

# 3. Remove rows with missing titles
all_titles_df = all_titles_df.dropna(subset=['title'])

# 4. Find titles that appear more than once
title_counts = all_titles_df['title'].value_counts()
duplicate_titles = title_counts[title_counts > 1].index.tolist()

# 5. Filter to show only duplicates and group by title
if not duplicate_titles:
    print("No duplicate titles found across the DataFrames.")
else:
    duplicates_df = all_titles_df[all_titles_df['title'].isin(duplicate_titles)]
    
    print(f"Found {len(duplicate_titles)} duplicate titles across datasets")
    print(f"Total duplicate entries: {len(duplicates_df)}")
    print("=" * 80)
    # Group by title and show which sources contain each duplicate
    for title in sorted(duplicate_titles):
        title_data = duplicates_df[duplicates_df['title'] == title]
        sources = title_data['source'].tolist()
        count = len(sources)

        
        print(f"\nTitle: {title}")
        print(f"   Total occurrences: {count}")
        print(f"   Found in: {', '.join(sources)}")

In [ ]:
# Standardize all dataframes to common schema: id, title, abstract, library, authors, doi, journal, date

# ACM
acm_df["library"] = "acm"
acm_df = acm_df.rename(columns={
    "ID": "id", "title": "title", "abstract": "abstract", "author": "authors",
    "doi": "doi", "journal": "journal", "year": "date"
})
for col in ["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]:
    if col not in acm_df.columns:
        acm_df[col] = pd.NA
acm_df = acm_df[["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]]

# Elsevier
elsevier_df["library"] = "elsevier"
elsevier_df = elsevier_df.rename(columns={
    "ID": "id", "title": "title", "abstract": "abstract", "author": "authors",
    "doi": "doi", "journal": "journal", "year": "date"
})
if "date" in elsevier_df.columns:
    elsevier_df["date"] = elsevier_df["date"].astype(str)
for col in ["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]:
    if col not in elsevier_df.columns:
        elsevier_df[col] = pd.NA
elsevier_df = elsevier_df[["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]]

# IEEE
ieee_df["library"] = "ieee"
ieee_df = ieee_df.rename(columns={
    "ISBNs": "id", "Document Title": "title", "Abstract": "abstract", "Authors": "authors",
    "DOI": "doi", "Publication Title": "journal", "Online Date": "date"
})
for col in ["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]:
    if col not in ieee_df.columns:
        ieee_df[col] = pd.NA
ieee_df = ieee_df[["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]]

# PubMed
pubmed_df["library"] = "pubmed"
pubmed_df = pubmed_df.rename(columns={
    "PMID": "id", "Title": "title", "abstract": "abstract", "Authors": "authors",
    "DOI": "doi", "Journal/Book": "journal", "Create Date": "date"
})
for col in ["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]:
    if col not in pubmed_df.columns:
        pubmed_df[col] = pd.NA
pubmed_df = pubmed_df[["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]]

# Springer - USE ACTUAL COLUMN NAMES from the parquet file
springer_df["library"] = "springer"
springer_df = springer_df.rename(columns={
    "Item Title": "title", "Abstract": "abstract", "Authors": "authors",
    "Item DOI": "doi", "Publication Title": "journal", "Publication Year": "date"
})
# Add ID if missing
if "id" not in springer_df.columns:
    springer_df["id"] = range(len(springer_df))
if "date" in springer_df.columns:
    springer_df["date"] = springer_df["date"].astype(str)
for col in ["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]:
    if col not in springer_df.columns:
        springer_df[col] = pd.NA
springer_df = springer_df[["id", "title", "abstract", "library", "authors", "doi", "journal", "date"]]


In [ ]:
print(f"ACM records after normalization: {len(acm_df)}")
print(f"IEEE records after normalization: {len(ieee_df)}")
print(f"PubMed records after normalization: {len(pubmed_df)}")
print(f"Elsevier records after normalization: {len(elsevier_df)}")
print(f"Springer records after normalization: {len(springer_df)}")
print(f"\nSpringer columns: {springer_df.columns.tolist()}")
print(f"Springer library value counts:\n{springer_df['library'].value_counts()}")


In [ ]:
combined_df = pd.concat([acm_df, elsevier_df, ieee_df, pubmed_df, springer_df], ignore_index=True)
print(len(combined_df))
combined_df
# print group count by date
date_counts = combined_df['date'].value_counts().sort_index()
print("Count of papers by date:")
date_counts

In [ ]:
print("Library counts immediately after concat:")
print(combined_df['library'].value_counts())
print(f"Springer count: {(combined_df['library'] == 'springer').sum()}")


In [ ]:
# Check library counts BEFORE duplicate removal
print("Library counts BEFORE duplicate removal:")
print(combined_df['library'].value_counts())
print(f"\nTotal records: {len(combined_df)}\n")

# create a normalized comparison key
combined_df['title_norm'] = combined_df['title'].str.lower().str.strip()

# group and filter to only duplicated titles
dupes = combined_df[combined_df.duplicated('title_norm', keep=False)]

# now show which libraries the duplicates belong to
result = dupes.groupby('title_norm')['library'].unique().reset_index()

print(result)


In [ ]:
# DROP DUPLICATES
combined_df['title_norm'] = combined_df['title'].str.lower().str.strip()

# find duplicated keys
dupe_keys = combined_df['title_norm'][combined_df['title_norm'].duplicated(keep=False)]

# remove every row with those keys
combined_df = combined_df[~combined_df['title_norm'].isin(dupe_keys)]

combined_df = combined_df.drop(columns='title_norm')
print(f"Library counts AFTER duplicate removal: {len(combined_df)}")
# remove journal is medRxiv
combined_df = combined_df[combined_df['journal'] != 'medRxiv']
print(f"Library counts AFTER removing medRxiv: {len(combined_df)}")
combined_df
print(f"Amount of papers after duplication removal: {len(combined_df)}")

# read my cache
combined_df = pd.read_parquet("dataset/combined_filtered.parquet")
combined_df
# save as csv
# sort by date_crossreq

combined_df = combined_df.sort_values(by='date_crossreq')

print(f"current date counts: {len(combined_df)}")

# filter to only papers with date between 2023 and 2025Q3
combined_df = combined_df[
    (combined_df['date_crossreq'] >= '2023-01-01') &
    (combined_df['date_crossreq'] <= '2025-09-30')
]
# filter date is empty
combined_df = combined_df[combined_df['date_crossreq'].notna()]
print(f"after filter date current date counts: {len(combined_df)}")
combined_df.to_csv("dataset/combined_filtered.csv", index=False)
# remove date which is not in 2023-2025Q3
combined_df.to_parquet("dataset/combined_filtered2.parquet", index=False)

In [ ]:
import matplotlib.pyplot as plt
# caculate group count by library, and draw a bar chart
library_counts = combined_df['library'].value_counts()
print(library_counts)
library_counts.plot(kind='bar', color="darkblue", title='Number of Records by Library in scientific research')


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# Combine all abstracts into one string
all_abstracts = ' '.join(combined_df['abstract'].dropna().astype(str))




# load stop words
my_stop_words = set(stopwords.words('english'))
# remove words like "using","there"
custom_stops = {"using", "there", "the", "and", "or", "but", "in", "on", "at", "to", "for", "of", "with","also","however","this","that","these","those"}
stop_words = my_stop_words.union(custom_stops)

# Create wordcloud
wordcloud = WordCloud(width=800, height=400, background_color='white',stopwords=my_stop_words).generate(all_abstracts)

# Display
# plt.figure(figsize=(12, 6))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')
# plt.title('Wordcloud of Abstracts for scientific research papers')
# plt.tight_layout(pad=0)
# this image doesn't need to be shown in the paper, so we can comment it out,please read wordcloud_generate_general.py for details
# plt.show()



In [ ]:
# group by journal and order by count descending
top15journal = combined_df.groupby('journal').size().sort_values(ascending=False).head(15)
# plot bar chart
top15journal.plot(kind='bar', color="darkblue", title='Top 15 Journals by Number of Publications in scientific research')





In [ ]:
import re

custom_regex_dict = {
    'chatgpt': re.compile(r'\b(chat[\s\-\.]*gpt)\b|\b(gpt)\b', re.IGNORECASE),
    'gpt-4': re.compile(r'\b(gpt[\s\-\.]*4(o|\s*turbo)?)\b', re.IGNORECASE),
    'gpt-3.5': re.compile(r'\b(gpt[\s\-\.]*3[\s\-\.]*5)\b', re.IGNORECASE),
    'gpt-5': re.compile(r'\b(gpt[\s\-\.]*5)\b', re.IGNORECASE), 
    'gpt-3': re.compile(r'\b(gpt[\s\-\.]*3)\b', re.IGNORECASE),
    'gemini': re.compile(r'\b(gemini)\b', re.IGNORECASE),
    'deepseek': re.compile(r'\b(deepseek)\b', re.IGNORECASE),
    'llama': re.compile(r'\b(llama)\b', re.IGNORECASE),
    'claude': re.compile(
        r'\b(claude)[\s\-\.]*(\(anthropic\)|\s*anthropic)?\b|\b(anthropic)\b', 
        re.IGNORECASE
    ),
    'mistral': re.compile(
    # Matches: 'mistral', 'mistral (le chat)', 'mistral le-chat'
    r'\b(mistral)[\s\-\.]*(\(le[\s\-\.]*chat\)|\s*le[\s\-\.]*chat)?\b'
    # OR Matches: 'le chat', 'le-chat', 'lechat' (requires 'le' and 'chat' together)
    r'|\b(le[\s\-\.]*chat)\b', 
    re.IGNORECASE
    ),
    'medgemma': re.compile(r'\b(med[\s\-\.]*gemma)\b', re.IGNORECASE),
    'gemma': re.compile(r'\b(gemma)\b', re.IGNORECASE),
    'med-palm': re.compile(r'\b(med[\s\-\.]*palm)\b', re.IGNORECASE), 
    'bert': re.compile(r'\b(bert)\b', re.IGNORECASE),
    'qwen': re.compile(r'\b(qwen)\b', re.IGNORECASE),
    'falcon': re.compile(r'\b(falcon)\b', re.IGNORECASE),
    'phi': re.compile(r'\b(phi)\b', re.IGNORECASE),
    'pubmedbert': re.compile(r'\b(pubmedbert)\b', re.IGNORECASE),
}
model_counts = {}

# Loop through each model and its regex
for model, regex in custom_regex_dict.items():
    count = combined_df["abstract"].str.count(regex).sum()
    model_counts[model] = int(count)

model_counts

In [ ]:
topic_keywords = ["performance", "method", "patient evaluated", "quality research"]

In [ ]:
def find_llms(text, regex_dict):
    """Finds all unique canonical LLMs that match a pattern in the text."""
    found_llms = []
    for llm_name, pattern in regex_dict.items():
        if pattern.search(text):
            found_llms.append(llm_name)
    return list(set(found_llms))

def find_topics(text, keywords):
    """Finds all unique keywords that appear in the text."""
    text_lower = text.lower()
    found_topics = []
    for keyword in keywords:
        # Simple substring search is sufficient since the topics are single words
        if keyword in text_lower:
            found_topics.append(keyword)
    return list(set(found_topics))

In [ ]:

# Apply the functions to the DataFrame
import numpy as np
import seaborn as sns
df = combined_df.copy()
df['Found_LLMs'] = df['abstract'].apply(
    lambda x: find_llms(x, custom_regex_dict) if pd.notna(x) else []
)

# Find Topics
df['Found_Topics'] = df['abstract'].apply(
    lambda x: find_topics(x, topic_keywords) if pd.notna(x) else []
)
# Define Matrix Axes
llm_labels = list(custom_regex_dict.keys())
topic_labels = topic_keywords

# Initialize the Occurrence Matrix
occurrence_matrix = pd.DataFrame(
    np.zeros((len(topic_labels), len(llm_labels)), dtype=int),
    index=topic_labels,
    columns=llm_labels
)

# Populate the occurrence matrix
for _, row in df.iterrows():
    llms_in_doc = row['Found_LLMs']
    topics_in_doc = row['Found_Topics']
    
    # Iterate over every combination of LLM and Topic found in the document
    for topic in topics_in_doc:
        for llm in llms_in_doc:
            # Increment the count where the topic and LLM co-occur
            occurrence_matrix.loc[topic, llm] += 1

In [ ]:
plt.figure(figsize=(14, 8))
sns.heatmap(
    occurrence_matrix,
    annot=True,        # Show the count in each cell
    cmap="Blues", # Reversed color map for better contrast
    linewidths=.5,     
    linecolor='black',
    fmt='d' # forces integer display
)
plt.title('Co-occurrence Matrix: LLMs vs. Application Topics in scientific research', fontsize=16)
plt.xlabel('Language Model', fontsize=14)
plt.ylabel('Application Topic', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:

df = combined_df.copy()
start = pd.Timestamp('2023-01-01')
cutoff = pd.Timestamp('2025-09-30')

df["date_standard"] = pd.to_datetime(df["date"], errors='coerce', dayfirst=True)
df = df[(df['date_standard'] >= start) & (df['date_standard'] <= cutoff)]




In [ ]:
# To make sure very paper is fouchs on LLMs & Health on scientific research, by using regular expression to filter the title and abstract
llm_pattern = re.compile(
    r'\b(chat[\s\-\.]*gpt|gpt[\s\-\.]*4(o|\s*turbo)?|gpt[\s\-\.]*3[\s\-\.]*5|gpt[\s\-\.]*5|gpt[\s\-\.]*3|gemini|deepseek|llama|claude[\s\-\.]*(\(anthropic\)|\s*anthropic)?|mistral[\s\-\.]*(\(le[\s\-\.]*chat\)|\s*le[\s\-\.]*chat)?|le[\s\-\.]*chat|med[\s\-\.]*gemma|gemma|med[\s\-\.]*palm|bert|qwen|falcon|phi|pubmedbert)\b', 
    re.IGNORECASE
)
health_pattern = re.compile(
    r'\b(health|healthcare|medicine|medical|clinical|patient|diagnosis|treatment|disease|symptom|hospital|doctor|nurse|pharmacy|biomedical|public health|epidemiology|mental health|health informatics|health data|health outcomes|health services|health policy|health management|health economics|health education|health promotion|health disparities|health equity|health behavior|health communication|health literacy|health intervention|health monitoring|health assessment|health research|health innovation|health technology|health system|healthcare delivery|healthcare management|healthcare policy|healthcare economics|healthcare education|healthcare promotion|healthcare disparities|healthcare equity|healthcare behavior|healthcare communication|healthcare literacy|healthcare intervention|healthcare monitoring|healthcare assessment|healthcare research|healthcare innovation|healthcare technology|healthcare system)\b', 
    re.IGNORECASE
)
scientific_research_pattern = re.compile(
    r'\b(scientific research|research|study|analysis|evaluation|investigation|experiment|clinical trial|systematic review|meta-analysis|case study|longitudinal study|cross-sectional study|observational study|randomized controlled trial|cohort study|qualitative research|quantitative research)\b', 
    re.IGNORECASE
)
def is_relevant_paper(title, abstract):
    text = f"{title} {abstract}"
    return bool(llm_pattern.search(text)) and bool(health_pattern.search(text)) and bool(scientific_research_pattern.search(text))
# Apply the relevance filter
df['is_relevant'] = df.apply(lambda row: is_relevant_paper(row['title'], row['abstract']), axis=1)
# Filter to only relevant papers
relevant_df = df[df['is_relevant']]
print(f"Number of papers after relevance filtering: {len(relevant_df)}")


In [ ]:
# Parse dates from the raw combined_df['date'] to avoid filtered-state issues
import re
from dateutil.parser import parse as dateutil_parse



# attach to a fresh copy and restrict to window
ndf = df.copy()
ndf['date_standard'] = ndf['date_crossreq']
start = pd.Timestamp('2023-01-01')
end = pd.Timestamp('2025-09-30')
ndf = ndf[(ndf['date_standard'] >= start) & (ndf['date_standard'] <= end)]

# quarterly
ndf['year_quarter'] = ndf['date_standard'].dt.to_period('Q')
quarter_counts = ndf['year_quarter'].value_counts().sort_index()
full_q = pd.period_range(start='2023Q1', end='2025Q3', freq='Q')
quarter_counts = quarter_counts.reindex(full_q, fill_value=0)

# show sample parsed rows that have non-January months
sample_non_jan = ndf.loc[ndf['date_standard'].dt.month != 1, ['date','date_standard']].head(30)
print('\nSample non-January parsed rows:')
print(sample_non_jan)

# plot
plt.figure(figsize=(12,6))
plt.bar([str(q) for q in full_q], quarter_counts.values, color='darkblue')
plt.xlabel('Quarter')
plt.ylabel('Count')
plt.title('Scientific Research Paper Counts by Quarter')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print('\nQuarter counts:')
print(quarter_counts)


In [ ]:
# Diagnostic: inspect rows with month names and how dateutil parses them
from dateutil.parser import parse as dateutil_parse
months_regex = r"(?i)\b(jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:t(?:ember)?)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\b"

mask = combined_df['date'].astype(str).str.contains(months_regex, regex=True, na=False)
sample = combined_df.loc[mask, ['library','date']].head(100).copy()

def try_parse(x):
    txt = str(x)
    try:
        return pd.Timestamp(dateutil_parse(txt, dayfirst=True, fuzzy=True))
    except Exception:
        try:
            return pd.Timestamp(dateutil_parse(txt, dayfirst=False, fuzzy=True))
        except Exception:
            return pd.NaT

sample['parsed'] = sample['date'].apply(try_parse)
print('Sample parsed results (month-name rows):')
print(sample.to_string())

# Show count of parsed months for month-name rows
print('\nParsed month value counts for these rows:')
print(sample['parsed'].dt.month.value_counts().sort_index())


In [ ]:
# Debug: inspect parsed months and sample raw date strings
print('Parsed date_standard month counts:')
print(df['date_standard'].dt.month.value_counts().sort_index())

print('\nSample raw -> parsed (first 50 non-null parsed):')
sample = df.loc[df['date_standard'].notna(), ['date','date_standard']].head(50)
for i, row in sample.iterrows():
    print(row['date'], '->', row['date_standard'])

print('\nExamples where month != 1 (should show Q2-Q4 entries):')
examples = df.loc[df['date_standard'].notna() & (df['date_standard'].dt.month != 1), ['date','date_standard']].head(50)
print(len(examples))
for i, row in examples.iterrows():
    print(row['date'], '->', row['date_standard'])


In [ ]:
# Search combined_df['date'] for month names / month-like strings
months_regex = r"(?i)\b(jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:t(?:ember)?)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\b"
mask = combined_df['date'].astype(str).str.contains(months_regex, regex=True, na=False)
print('Records with month names in `date`:', mask.sum())
print('\nSample rows with month in date:')
print(combined_df.loc[mask, ['library','date']].head(40))

# Also check for formats like '2025-September' or '2024-09'
mask2 = combined_df['date'].astype(str).str.contains(r'\d{4}[-_/]\s*[A-Za-z]+|\d{4}[-_/]\s*\d{1,2}', regex=True, na=False)
print('\nRecords matching year-month patterns:', mask2.sum())
print(combined_df.loc[mask2, ['library','date']].head(40))


In [ ]:
combined_df

print(f"Total papers before filtering: {len(combined_df)}")

# 1. define regular expression for filtering section 3.7
keywords = [
    "automated systematic review", "automated review", "evidence synthesis",
    "hypothesis generation", "scientific discovery", "AI scientist",
    "CRISPR-GPT", "FutureHouse", "Laser AI", "gene editing",
    "experimental design", "troubleshooting protocols", "autonomous agents","review", "systematic review", "literature review", "meta-analysis", "research synthesis", "research evaluation", "research assessment", "research quality", "research performance", "patient evaluated", "quality research"
]

# 2. build combined regex pattern
regex_pattern = '|'.join(keywords)

# 3. filter abstracts using the regex pattern
# na=False filters out NaN abstracts
gaosong_papers = combined_df[combined_df['abstract'].str.contains(regex_pattern, case=False, na=False)]

# 4. if needed, further filter for LLM mentions
# gaosong_papers = gaosong_papers[gaosong_papers['abstract'].str.contains("LLM|Large Language Model", case=False, na=False)]
print(f"papers filter abstracts using the regex pattern: {len(gaosong_papers)}")
print(f"筛选出的论文数量: {len(gaosong_papers)}")
gaosong_papers

# filter papers that abstract contains "CRISPR-GPT"
crispr_gpt_papers = combined_df[combined_df['abstract'].str.contains("CRISPR", case=False, na=False)]
print(f"筛选出的CRISPR-GPT相关论文数量: {len(crispr_gpt_papers)}")
# only print titile and abstract(no truncate) for crispr_gpt_papers

# print title and abstract without truncation
pd.set_option('display.max_colwidth', None)
crispr_gpt_papers.to_csv("crispr_gpt_papers.csv", index=False)
# reset display option
pd.reset_option('display.max_colwidth')